# Riesgo Operacional
---

## Definiciones

**Definición Global (BIS)**: "El riesgo de pérdida resultante de la inadecuación o fallos de los procesos internos, el personal y los sistemas, o bien de acontecimientos externos." (Basel II).


## **Clasificación de eventos de pérdida  (Basel II)**

Los 4 Factores de de Eventos de Pérdida:

In [ ]:
# @title
from IPython.display import Markdown
Markdown(f"""
<img src="https://ecampusontario.pressbooks.pub/app/uploads/sites/4230/2024/05/Categories-of-operational-risk-1536x771.png"
     width="700"
     alt="Categorías de Riesgo Operacional (Basel)">
""")


<img src="https://ecampusontario.pressbooks.pub/app/uploads/sites/4230/2024/05/Categories-of-operational-risk-1536x771.png" 
     width="700" 
     alt="Categorías de Riesgo Operacional (Basel)">


Las 7 categorías de Eventos de Pérdida:

| Categoría (Basilea II) | Ejemplos | Frecuencia (General) | Severidad (General) |
| :--- | :--- | :--- | :--- |
| 1. Fraude Interno | Kerviel, apropiación de fondos por un cajero | Baja | Alta |
| 2. Fraude Externo | Ciberataques, phishing, fraude con tarjetas de crédito | Alta | Media |
| 3. Prácticas Laborales | Demandas por discriminación, accidentes laborales | Media | Media |
| 4. Clientes, Productos |  Venta indebida de productos, fallos en protección de datos del cliente | Alta | Media-Alta |
| 5. Daños a Activos Físicos | Incendios, terremotos, inundación del centro de dato | Baja | Alta |
| 6. Interrupción de Negocio | Caída del core bancario, fallos en la plataforma de trading | Media | Alta |
| 7. Fallos de Proceso | Errores en captura de datos, transacciones no liquidadas, documentación legal defectuosa | Alta | Baja |


--------------------------------------------------------------------------------

**El desafío no es la clasificación, sino la recolección de datos (LDC)**. El problema del "iceberg": solo se reportan las pérdidas grandes, pero las pequeñas y frecuentes (y los casi-incidentes o near-misses) son más predictivas y cruciales para la gestión.

**ALGUNOS CASOS DE ESTUDIO**:

* Société Générale (2008) – Fraude interno → €4.9 mil millones
* Knight Capital (2012) – Error de software → \$460 millones en 45 min
* Banco Popular (Colombia, 2020) – Ciberataque ransomware → \$12 millones en pérdidas operacionales


## Modelos de RO

**Basel II permite 3 enfoques:**

**1 . Indicador Básico (BIA) → $\alpha: 15\%$ de ingresos brutos promedio**

$$Capital_{RO} = \alpha \times \text{Promedio(Ingresos Brutos últimos 3 años)}$$


**2 . Enfoque Estándar (TSA) → x% variable por línea de negocio**

$$Capital_{RO} = \sum_{i=1}^{8} \beta_i \times IB_i \text{(por línea de negocio)}$$

| **Línea de Negocio** | **β (%)** | **Descripción breve** |
|----------------------|----------|------------------------|
| **Financiamiento Corporativo** *(Corporate Finance)* | **18** | M&A, sindicación, emisiones de valores, asesoría financiera. |
| **Trading & Sales** | **18** | Operaciones en mercados (acciones, renta fija, derivados, FX). |
| **Banca Minorista** *(Retail Banking)* | **12** | Cuentas, tarjetas, préstamos personales, hipotecas. |
| **Banca Comercial** *(Commercial Banking)* | **15** | Préstamos a PYMES, líneas de crédito, factoring. |
| **Procesamiento de Pagos y Liquidación** *(Payment & Settlement)* | **18** | Transferencias, compensación, sistemas de pago (ACH, SWIFT). |
| **Servicios de Agencia** *(Agency Services)* | **15** | Custodia, administración de fondos, escrow. |
| **Gestión de Activos** *(Asset Management)* | **12** | Fondos mutuos, carteras discrecionales, ETF. |
| **Intermediación Minorista** *(Retail Brokerage)* | **12** | Ejecución de órdenes para clientes minoristas. |

--------------------------------------------------------------------------------



**3 . Enfoque Avanzado (AMA) → Modelos internos (LDA)**

Aquí es donde debemos enfocarnos.

* El LDA permite a las entidades usar sus propios modelos internos.
* El estándar de facto fue el **Loss Distribution Approach** (LDA).

$$Capital_{RO} = VaR_{99.9\%} (\text{Pérdida Agregada Anual})$$

### Enfoque AMA - Loss Distribution Approach (LDA)

Este es el modelo que se implementará.

1. Pérdida Agregada (L): Es la suma de todas las pérdidas en un año.

2. Componentes del LDA: Se modelan dos cosas por separado:

* Frecuencia (N): ¿Cuántos eventos de pérdida ocurren en un año? Distribución Discreta: Típicamente Distribución de Poisson.

* Severidad (X): ¿Cuál es el monto de cada pérdida individual? Distribución Continua: Debe tener "cola pesada" (fat tail). Típicamente Lognormal o Pareto.

$$ \text{Pérdida Agregada (L)}: L = \sum_{i=1}^{N} X_i $$

El Desafío: No hay una solución analítica (fórmula cerrada) para la distribución de $L$.

La Solución: **Simulación de Monte Carlo**.

## BIBLIOGRAFÍA OBLIGATORIA

Moosa, I. (2007). Operational Risk Management. Palgrave Macmillan. Cap. 4 y 5

Hull, J. (2015). Risk Management and Financial Institutions. Wiley. Cap. 20

BCBS (2004). Basel II: International Convergence of Capital Measurement.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Cálculo BIA vs TSA

# Datos del banco (en millones de COP)
ingresos = {
    'Banca Corporativa': 120000,
    'Banca Minorista': 80000,
    'Trading': 30000,
    'Pagos': 15000
}

#  BIA
ingreso_total = sum(ingresos.values())
capital_bia = ingreso_total * 0.15
print(f"Capital por BIA: {capital_bia:,.0f} millones")

Capital por BIA: 36,750 millones


In [ ]:
# Factores Basel TSA
factores_tsa = {
    'Banca Corporativa': 0.18,
    'Banca Minorista': 0.12,
    'Trading': 0.18,
    'Pagos': 0.18
}

# TSA
capital_tsa = sum(ingresos[linea] * factores_tsa[linea] for linea in ingresos)
print(f"Capital por TSA: {capital_tsa:,.0f} millones")

Capital por TSA: 39,300 millones


In [ ]:
# Gráfico
ca_ro = pd.DataFrame({
    'Enfoque': ['BIA', 'TSA'],
    'Capital Requerido': [capital_bia, capital_tsa]
})

ca_ro

,Enfoque,Capital Requerido
0,BIA,36750.0
1,TSA,39300.0
